In [ ]:
import cv2
import os
from pathlib import Path
from matplotlib import pyplot as plt
import math
import numpy as np
from tqdm import tqdm

In [ ]:
path = Path("Dataset_TP3")
videos_paths = [p for p in path.iterdir() if p.is_file()]

videos: dict[Path, cv2.VideoCapture] = {}
for file in videos_paths:
    vid = cv2.VideoCapture(file)
    if not vid.isOpened():
        print("Error: Could not open video.")
        continue
    videos[file] = vid
    break

In [ ]:
interactive = False
r = 50  

edges = {}

print(f"Processing {len(videos)} videos")
for video_path in videos:
    video = videos[video_path]
    fps = video.get(cv2.CAP_PROP_FPS)
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    distances = None

    for frame_idx in tqdm(range(frame_count)):
        edges_lst = []
        ret, frame = video.read()
        if not ret:
            print("Error: Could not read frame.")
            break


        # Dilatation des gradients en utilisant un noyau circulaire
        if distances is None:
            h, w = sobel_x.shape
            center_y, center_x = h // 2, w // 2
            y, x = np.ogrid[:h, :w]
            distances = np.sqrt((x - center_x) ** 2 + (y - center_y) ** 2)
            max_dist = np.max(distances)
            bin_size = max_dist / r
        
        
        sobel_x_reshaped = np.zeros(r)
        sobel_y_reshaped = np.zeros(r)
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
        sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)

        for i in range(r):
            mask = (distances >= i * bin_size) & (distances < (i + 1) * bin_size)
            if np.any(mask):
                sobel_x_reshaped[i] = np.mean(np.abs(sobel_x[mask]))
                sobel_y_reshaped[i] = np.mean(np.abs(sobel_y[mask]))

        edges_lst.append((sobel_x_reshaped, sobel_y_reshaped))

        if interactive:
            magnitude = cv2.magnitude(sobel_x, sobel_y)
            edged = cv2.convertScaleAbs(magnitude)
            cv2.imshow("Edged Frame", edged)
            if cv2.waitKey(int(1000 / fps)) & 0xFF == ord("q"):
                break

    edges[video_path] = edges_lst

In [ ]:
for video_name in edges:
    edges_lst = edges[video_name]
    for sobel_x, sobel_y in edges_lst:
        for i in range(len(sobel_x) - 1):
            # current_x = sobel_x[i]
            # current_y = sobel_y[i]
            # future_x = sobel_x[i + 1]
            # future_y = sobel_y[i + 1]
            # pIn = 1-  ((current_x + future_x) * (current_y + future_y)) / (future_x * future_y)
            # pOut = 1 - (())
            print(f"pIn: {pIn}")
        break